In [245]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from langgraph.types import CachePolicy
from langgraph.cache.memory import InMemoryCache
from datetime import datetime
from typing import Annotated

In [246]:
class PrivateState(TypedDict):
    a: int
    b: int


def update_function(old, new):
    return old + new


class InputState(TypedDict):
    hello: Annotated[list[str], update_function]


class OutputState(TypedDict):
    bye: Annotated[list[str], update_function]


graph_builder = StateGraph(InputState)


In [247]:
def node_one(state: InputState) -> InputState:
    print("node1: ", state)
    return {
        "hello": ["node1"],
    }


def node_two(state: InputState) -> InputState:
    print("node2: ", state)
    return {
        "hello": [f"{datetime.now()}"],
    }


def node_three(state: InputState) -> InputState:
    print("node3: ", state)
    return {
        "hello": ["node3"],
    }


def node_four(state: InputState) -> InputState:
    print("node4: ", state)
    return {
        "hello": ["node4"],
    }

In [248]:
graph_builder.add_node("node_one", node_one)
graph_builder.add_node(
    "node_two",
    node_two,
    cache_policy=CachePolicy(ttl=20)
)
graph_builder.add_node("node_three", node_three)
graph_builder.add_node("node_four", node_four)

graph_builder.add_edge(START, "node_one")
graph_builder.add_edge("node_one", "node_two")
graph_builder.add_edge("node_two", "node_three")
graph_builder.add_edge("node_three", "node_four")
graph_builder.add_edge("node_four", END)


In [249]:
graph = graph_builder.compile(cache=InMemoryCache())
# display(graph)
# print(graph.get_graph().draw_ascii())



import time

result = graph.invoke({
    "hello": ["world"],
})
time.sleep(5)
result = graph.invoke({
    "hello": ["world"],
})
time.sleep(5)
result = graph.invoke({
    "hello": ["world"],
})
time.sleep(5)
result = graph.invoke({
    "hello": ["world"],
})
time.sleep(5)
result = graph.invoke({
    "hello": ["world"],
})
time.sleep(5)
result = graph.invoke({
    "hello": ["world"],
})
time.sleep(5)
result = graph.invoke({
    "hello": ["world"],
})
time.sleep(5)
print("result:", result)

node1:  {'hello': ['world']}
node2:  {'hello': ['world', 'node1']}
node3:  {'hello': ['world', 'node1', '2025-10-10 12:49:28.201033']}
node4:  {'hello': ['world', 'node1', '2025-10-10 12:49:28.201033', 'node3']}
node1:  {'hello': ['world']}
node3:  {'hello': ['world', 'node1', '2025-10-10 12:49:28.201033']}
node4:  {'hello': ['world', 'node1', '2025-10-10 12:49:28.201033', 'node3']}
node1:  {'hello': ['world']}
node3:  {'hello': ['world', 'node1', '2025-10-10 12:49:28.201033']}
node4:  {'hello': ['world', 'node1', '2025-10-10 12:49:28.201033', 'node3']}
node1:  {'hello': ['world']}
node3:  {'hello': ['world', 'node1', '2025-10-10 12:49:28.201033']}
node4:  {'hello': ['world', 'node1', '2025-10-10 12:49:28.201033', 'node3']}
node1:  {'hello': ['world']}
node2:  {'hello': ['world', 'node1']}
node3:  {'hello': ['world', 'node1', '2025-10-10 12:49:48.216397']}
node4:  {'hello': ['world', 'node1', '2025-10-10 12:49:48.216397', 'node3']}
node1:  {'hello': ['world']}
node3:  {'hello': ['world